In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler , MinMaxScaler
import time

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , LSTM , Dropout

t_0 = time.time()
data = pd.read_csv (r'train.csv',nrows=1000000)   #,nrows=
t_1 = time.time()
print(f'Time : {t_1-t_0}')

Time : 101.0259006023407


In [46]:
training_data = data[data['investment_id']<800]
test_data = data[(data['investment_id']>=800) & (data['investment_id']<=1000)]

training_data = training_data.drop(['row_id' , 'time_id' , 'investment_id' , 'target'], axis=1)
test_data = test_data.drop(['row_id' , 'time_id' , 'investment_id' , 'target'], axis=1)
training_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [47]:
x_train = []
y_train = []

In [48]:
for j in range(training_data.shape[0]):
    for i in range(299,training_data.shape[1]):
        x_train.append(training_data.loc[j][i-299:i])
        y_train.append(training_data.loc[j][i])

In [49]:
x_train , y_train = np.array(x_train) , np.array(y_train)

In [50]:
from numpy import zeros, newaxis
x_train = x_train[:, :, newaxis]

In [51]:
x_train.shape , y_train.shape , type(x_train) , type(y_train)

((212500, 299, 1), (212500,), numpy.ndarray, numpy.ndarray)

In [52]:
regressior = Sequential()
               
regressior.add(LSTM(units = 50, activation = 'sigmoid', return_sequences = True, input_shape=(x_train.shape[1:])))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 60, activation = 'tanh', return_sequences = True))
regressior.add(Dropout(0.3))

regressior.add(LSTM(units = 80, activation = 'sigmoid', return_sequences = True))
regressior.add(Dropout(0.4))

regressior.add(LSTM(units = 120, activation = 'tanh', ))
regressior.add(Dropout(0.5))

regressior.add(Dense(units = 1))

In [53]:
regressior.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 299, 50)           10400     
_________________________________________________________________
dropout_12 (Dropout)         (None, 299, 50)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 299, 60)           26640     
_________________________________________________________________
dropout_13 (Dropout)         (None, 299, 60)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 299, 80)           45120     
_________________________________________________________________
dropout_14 (Dropout)         (None, 299, 80)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 120)              

In [54]:
regressior.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
t_0 = time.time()
regressior.fit(x_train, y_train, epochs=10, batch_size=32)
t_1 = time.time()
print(f'Time : {t_1-t_0}')

Epoch 1/10
5226/6641 [======================>.......] - ETA: 54:27 - loss: 0.6066

In [ ]:
x_test = []
y_test = []

In [ ]:
for j in range(test_data.shape[0]):
    for i in range(299,test_data.shape[1]):
        x_test.append(test_data.loc[j][i-299:i])
        y_test.append(test_data.loc[j][i])

In [ ]:
x_test , y_test = np.array(x_test) , np.array(y_test)

In [ ]:
x_test = x_test[:, :, newaxis]

In [ ]:
x_test.shape , y_test.shape , type(x_test) , type(y_test)

In [ ]:
y_pred = regressior.predict(x_test)

In [ ]:
def graph(list):
    graph = []
    for i in range(len(list)):
        if i == 0:
            graph.append(list[0])
        else:
            graph.append(graph[-1]+list[i])
    return(graph)

In [ ]:
y_pred = list(y_pred)
y_test = list(y_test)

In [ ]:
y_test_graph = graph(y_test)
y_pred_graph = graph(y_pred)

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test_graph,color='red')
plt.plot(y_pred_graph,color='blue')
plt.show()

In [ ]:
error = []
for i in range(len(y_pred)):
    err = abs(y_pred[i]-y_test[i])
    error.append(err)
f_error = sum(error)/len(y_pred)
print(f_error)

In [ ]:
[0.7652131]